### History based Linear model
- Miller et al. 2021

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from pathlib import Path
import pandas as pd
from numpy.lib.stride_tricks import sliding_window_view
import matplotlib.pyplot as plt
from neuropy import plotting
from scipy import stats


basepath = Path("D:\\Data")
# files = ["gronckle.csv", "grump.csv"]
files = sorted(basepath.glob("*.csv"))

fig = plotting.Fig(6, 3, size=(12, 5), num=1)

npast = 10
params_pooled = []
task_type_bool = []

for i, file in enumerate(files):
    data_df = pd.read_csv(basepath / file)
    prob_corr = np.abs(
        stats.pearsonr(data_df["rewprobfull1"], data_df["rewprobfull2"])[0]
    )

    task_type = "unstructured" if prob_corr < 0.2 else "structured"
    task_type_bool.append(prob_corr)

    choices = data_df["port"].to_numpy()
    choices[choices == 2] = -1
    outcomes = data_df["reward"].to_numpy()
    outcomes[outcomes == 0] = -1
    n_trials = choices.size

    past_choices = sliding_window_view(choices, npast)[:-1, :]
    past_outcomes = sliding_window_view(outcomes, npast)[:-1, :]
    actual_choices = choices[npast:]

    x = np.hstack(
        (
            past_choices * past_outcomes,
            past_choices,
            past_outcomes,
        )
    )
    clf = LogisticRegression(random_state=0).fit(x, actual_choices)

    params = np.fliplr(clf.coef_.squeeze().reshape(3, npast))
    params_pooled.append(params)

    subfig = fig.add_subfigure(fig.gs[i])
    subfig.suptitle(f"{files[i].name[:-4]}, {task_type}")
    sub_axs = subfig.subplots(1, 3, width_ratios=[1, 1, 1], sharey=True, sharex=True)

    colors = ["orange", "purple", "blue"]
    titles = ["Reward Seeking", "Choice Preservation", "Main effect of Outcome"]
    for _, ax in enumerate(sub_axs):

        ax.plot(np.arange(1, 11), params[_], ".-", color=colors[_], zorder=1)
        ax.set_title(titles[_])
        ax.axhline(0, color="gray", zorder=0, lw=0.8)
        ax.set_xticks([1, 5, 10])

    if i == 0:
        sub_axs[0].set_xlabel("Trials in the past")
        sub_axs[0].set_ylabel("Influence on current choice")

task_type_bool = np.array(task_type_bool)
params_pooled = np.array(params_pooled)
mean_struc = params_pooled[task_type_bool < 0.2, :, :].mean(axis=0)
mean_unstruc = params_pooled[task_type_bool > 0.2, :, :].mean(axis=0)

subfig = fig.add_subfigure(fig.gs[4:, 0:2])
subfig.suptitle(f"Mean across animals by task type")
sub_axs = subfig.subplots(1, 3, width_ratios=[1, 1, 1], sharey=True, sharex=True)

# colors = ["orange", "purple", "blue"]
colors = ["#5040BF", "#AFBF40"]


titles = ["Reward Seeking", "Choice Preservation", "Main effect of Outcome"]
for _, ax in enumerate(sub_axs):

    ax.plot(np.arange(1, 11), mean_struc[_], ".-", color=colors[0], alpha=0.7, zorder=1)
    ax.plot(
        np.arange(1, 11), mean_unstruc[_], ".-", color=colors[1], alpha=0.7, zorder=1
    )
    ax.legend(["Struc", "Unstruc"])
    ax.set_title(titles[_])
    ax.axhline(0, color="gray", zorder=0, lw=0.8)
    ax.set_xticks([1, 5, 10])

### Estimating Qlearning parameters (2 $\alpha$ model)
- Estimate learning parameter for chosen and unchosen arms/choices. If the animals are learning, then one should expect that the chosen and unchosen arm will have postive and negative $\alpha$ respectively.

In [ ]:
import numpy as np
import pandas as pd
from banditpy.analyses import QlearningEstimator
import mab_subjects

exps = mab_subjects.struc.allsess + mab_subjects.unstruc.allsess

params_df = []

for i, exp in enumerate(exps):
    mab = exp.mab.keep_by_trials(min_trials=100, clip_max=100)
    print(exp.sub_name)
    qlearn = QlearningEstimator(mab)
    qlearn.fit(
        x0=None,
        bounds=np.array([(-1, 1), (-1, 1), (0.005, 20)]),
        method="diff_evolution",
        n_opts=5,
        n_cpu=4,
    )

    qlearn.print_params()
    df = pd.DataFrame(
        {
            "name": exp.sub_name,
            "param": ["alpha_chosen", "alpha_unchosen", "beta"],
            "param_values": np.array([qlearn.alpha_c, qlearn.alpha_u, qlearn.beta]),
            "grp": "struc" if mab.is_structured else "unstruc",
        }
    )
    params_df.append(df)

params_df = pd.concat(params_df, ignore_index=True)
mab_subjects.GroupData().save(params_df, "qlearning_2alpha_params_anirudh")

In [ ]:
from neuropy import plotting
import pandas as pd
import seaborn as sns

params_df = mab_subjects.GroupData().qlearning_2alpha_params_anirudh

fig = plotting.Fig(1, 2, size=(4, 3), num=1)
ax1 = fig.subplot(fig.gs[0])
ax2 = fig.subplot(fig.gs[1])

plot_kw = dict(x="param", y="param_values", hue="grp")
bar_kw = dict(
    errorbar="se",
    palette="dark:black",
    linestyle="none",
    alpha=0.5,
    dodge=0.4,
    zorder=1,
    marker=".",
    markersize=10,
    markeredgewidth=0,
    err_kws={"linewidth": 1},
)
strip_kw = dict(palette="husl", alpha=0.5, dodge=True, zorder=2)


alpha_df = params_df[params_df["param"] != "beta"]
sns.pointplot(alpha_df, ax=ax1, **plot_kw, **bar_kw)
sns.stripplot(alpha_df, ax=ax1, **plot_kw, **strip_kw)

beta_df = params_df[params_df["param"] == "beta"]
sns.pointplot(beta_df, ax=ax2, **plot_kw, **bar_kw)
sns.stripplot(beta_df, ax=ax2, **plot_kw, **strip_kw)

ax1.set_ylabel("Estimated alpha values")
ax2.set_ylabel("Estimated beta values")
fig.fig.suptitle("Q-learning in two-armed bandit task")

### Qlearning with perseverance and scaler params
- When we estimated alpha parameters for chosen and unchosen choices, we found that unstructured env had higher alpha values for chosen arms compared to structured env. So we asked if 'persevrance' for arms/choices is making alpha_chosen higher for unstructured environment.

In [ ]:
import numpy as np
import pandas as pd
from banditpy.analyses import QlearningEstimator
import mab_subjects

exps = mab_subjects.struc.allsess + mab_subjects.unstruc.allsess

params_df = []

for i, exp in enumerate(exps):
    mab = exp.mab.keep_by_trials(min_trials=100, clip_max=100)
    print(exp.sub_name)
    qlearn = QlearningEstimator(mab, model="persev")
    qlearn.fit(
        x0=None,
        bounds=np.array([(-1, 1), (-1, 1), (0, 1), (1, 10), (0.005, 20)]),
        method="diff_evolution",
        n_opts=5,
        n_cpu=4,
    )

    qlearn.print_params()
    df = pd.DataFrame(
        {
            "name": exp.sub_name,
            "param": ["alpha_chosen", "alpha_unchosen", "persev", "scaler", "beta"],
            "param_values": qlearn.estimated_params,
            "grp": "struc" if mab.is_structured else "unstruc",
        }
    )
    params_df.append(df)

params_df = pd.concat(params_df, ignore_index=True)
mab_subjects.GroupData().save(params_df, "qlearning_2alpha_persev_anirudh")

In [27]:
from neuropy import plotting
import pandas as pd
import seaborn as sns
from statannotations.Annotator import Annotator
from statannotations.stats.StatTest import StatTest
from statplot_utils import stat_kw

params_df = mab_subjects.GroupData().qlearning_2alpha_persev_anirudh

fig = plotting.Fig(1, 3, size=(6, 4), num=1)
ax1 = fig.subplot(fig.gs[0])
ax2 = fig.subplot(fig.gs[1])

plot_kw = dict(x="param", y="param_values", hue="grp", hue_order=["unstruc", "struc"])
bar_kw = dict(
    errorbar="se",
    palette="dark:black",
    linestyle="none",
    alpha=0.5,
    dodge=0.4,
    zorder=1,
    marker=".",
    markersize=10,
    markeredgewidth=0,
    err_kws={"linewidth": 1},
)
strip_kw = dict(palette="husl", alpha=0.5, dodge=True, zorder=2)

indx_bool = params_df["param"].isin(["scaler", "beta"])

alpha_df = params_df[~indx_bool]
sns.pointplot(alpha_df, ax=ax1, **plot_kw, **bar_kw)
sns.stripplot(alpha_df, ax=ax1, **plot_kw, **strip_kw)

orders = ["alpha_chosen", "alpha_unchosen", "persev"]
pairs = [((_, "unstruc"), (_, "struc")) for _ in orders]
annotator = Annotator(pairs=pairs, data=alpha_df, ax=ax1, **plot_kw, order=orders)
annotator.configure(test="t-test_ind", **stat_kw, color="k", verbose=True)
annotator.apply_and_annotate()
annotator.reset_configuration()


beta_df = params_df[indx_bool]
sns.pointplot(beta_df, ax=ax2, **plot_kw, **bar_kw)
sns.stripplot(beta_df, ax=ax2, **plot_kw, **strip_kw)

orders = ["scaler", "beta"]
pairs = [((_, "unstruc"), (_, "struc")) for _ in orders]
annotator = Annotator(pairs=pairs, data=beta_df, ax=ax2, **plot_kw, order=orders)
annotator.configure(test="t-test_ind", **stat_kw, color="k", verbose=True)
annotator.apply_and_annotate()
annotator.reset_configuration()


ax1.axhline(0, ls="--", color="gray", zorder=0, lw=0.8)
ax2.set_xlim(-1, 2)
ax2.set_ylim(1, 15)

ax1.tick_params(axis="x", rotation=30)
ax2.tick_params(axis="x", rotation=30)

ax1.set_xlabel("")
ax2.set_xlabel("")

ax1.legend_.remove()
ax2.legend_.remove()
ax1.set_ylabel("Estimated alpha values")
ax2.set_ylabel("Estimated beta values")
fig.fig.suptitle("Q-learning in two-armed bandit task")

p-value annotation legend:
      ns: 5.00e-02 < p <= 1.00e+00
       *: p <= 5.00e-02

alpha_unchosen_unstruc vs. alpha_unchosen_struc: t-test independent samples, P_val:4.110e-01 t=8.621e-01
alpha_chosen_unstruc vs. alpha_chosen_struc: t-test independent samples, P_val:1.004e-03 t=4.778e+00
persev_unstruc vs. persev_struc: t-test independent samples, P_val:1.128e-02 t=-3.175e+00
p-value annotation legend:
      ns: 5.00e-02 < p <= 1.00e+00
       *: p <= 5.00e-02

beta_unstruc vs. beta_struc: t-test independent samples, P_val:5.252e-01 t=-6.609e-01
scaler_unstruc vs. scaler_struc: t-test independent samples, P_val:2.091e-01 t=-1.353e+00


Text(0.5, 0.98, 'Q-learning in two-armed bandit task')

### Params estimation for "Structured" sessions in Unstructured Environment.
- Basically select sessions where the probabilities sum up to 1 i.e, correlated sessions withing Unstructured environment. Estimate various params like alpha_chosen, alpha_unchosen etc. on these sessions and compare these with Structured Environment.

In [16]:
import numpy as np
import pandas as pd
from banditpy.analyses import QlearningEstimator
import mab_subjects

exps = mab_subjects.unstruc.allsess

params_df = []

for i, exp in enumerate(exps):
    mab = exp.mab.keep_by_trials(min_trials=100, clip_max=100)
    print(exp.sub_name)
    session_prob_sum = mab.probs[mab.is_session_start.astype(bool)].sum(axis=1)
    good_sessions = mab.sessions[session_prob_sum == 1]

    mab = mab.keep_sessions_by_id(good_sessions)

    qlearn = QlearningEstimator(mab, model="persev")
    qlearn.fit(
        x0=None,
        bounds=np.array([(-1, 1), (-1, 1), (0, 1), (1, 10), (0.005, 20)]),
        method="diff_evolution",
        n_opts=5,
        n_cpu=4,
    )

    qlearn.print_params()
    df = pd.DataFrame(
        {
            "name": exp.sub_name,
            "param": ["alpha_chosen", "alpha_unchosen", "persev", "scaler", "beta"],
            "param_values": qlearn.estimated_params,
            "grp": "struc" if mab.is_structured else "unstruc",
        }
    )
    params_df.append(df)

params_df = pd.concat(params_df, ignore_index=True)
mab_subjects.GroupData().save(
    params_df, "qlearning_2alpha_persev_correlated_within_unstructured_anirudh"
)

AggroExp1UnStructured
alpha_c: 0.0726, alpha_u: -0.0195,alpha_h: 0.1602, scaler: 3.2435, beta: 5.9231
AuromaExp1Unstructured
alpha_c: 0.1412, alpha_u: -0.116,alpha_h: 0.7797, scaler: 6.9614, beta: 3.4484
BratExp1Unstructured
alpha_c: 0.2178, alpha_u: -0.0868,alpha_h: 0.5768, scaler: 4.1272, beta: 1.0058
GronckleExp2Unstructured
alpha_c: 0.0257, alpha_u: -0.0045,alpha_h: 0.5257, scaler: 5.5276, beta: 8.3728
GrumpExp1Unstructured
alpha_c: -0.0248, alpha_u: -0.1555,alpha_h: 0.4394, scaler: 2.3295, beta: 3.4587
ToothlessExp2Unstructured
alpha_c: 0.0723, alpha_u: -0.0611,alpha_h: 0.8443, scaler: 8.4889, beta: 3.8096
qlearning_2alpha_persev_correlated_within_unstructured_anirudh saved


In [25]:
from neuropy import plotting
import pandas as pd
import seaborn as sns
from statannotations.Annotator import Annotator
from statannotations.stats.StatTest import StatTest
from statplot_utils import stat_kw

df1 = mab_subjects.GroupData().qlearning_2alpha_persev_anirudh
df2 = (
    mab_subjects.GroupData().qlearning_2alpha_persev_correlated_within_unstructured_anirudh
)
df2["grp"] = "struc_in_unstruc"
params_df = pd.concat([df1, df2], ignore_index=True)

fig = plotting.Fig(1, 3, size=(6, 4), num=1)
ax1 = fig.subplot(fig.gs[0])
ax2 = fig.subplot(fig.gs[1])

plot_kw = dict(
    x="param",
    y="param_values",
    hue="grp",
    hue_order=["unstruc", "struc", "struc_in_unstruc"],
)
bar_kw = dict(
    errorbar="se",
    palette="dark:black",
    linestyle="none",
    alpha=0.5,
    dodge=0.4,
    zorder=1,
    marker=".",
    markersize=10,
    markeredgewidth=0,
    err_kws={"linewidth": 1},
)
strip_kw = dict(palette="husl", alpha=0.5, dodge=True, zorder=2)

indx_bool = params_df["param"].isin(["scaler", "beta"])

alpha_df = params_df[~indx_bool]
sns.pointplot(alpha_df, ax=ax1, **plot_kw, **bar_kw)
sns.stripplot(alpha_df, ax=ax1, **plot_kw, **strip_kw)

# orders = ["alpha_chosen", "alpha_unchosen", "persev"]
# pairs = [((_, "unstruc"), (_, "struc")) for _ in orders]
# annotator = Annotator(pairs=pairs, data=alpha_df, ax=ax1, **plot_kw, order=orders)
# annotator.configure(test="t-test_ind", **stat_kw, color="k", verbose=True)
# annotator.apply_and_annotate()
# annotator.reset_configuration()


beta_df = params_df[indx_bool]
sns.pointplot(beta_df, ax=ax2, **plot_kw, **bar_kw)
sns.stripplot(beta_df, ax=ax2, **plot_kw, **strip_kw)

# orders = ["scaler", "beta"]
# pairs = [((_, "unstruc"), (_, "struc")) for _ in orders]
# annotator = Annotator(pairs=pairs, data=beta_df, ax=ax2, **plot_kw, order=orders)
# annotator.configure(test="t-test_ind", **stat_kw, color="k", verbose=True)
# annotator.apply_and_annotate()
# annotator.reset_configuration()


ax1.axhline(0, ls="--", color="gray", zorder=0, lw=0.8)
ax2.set_xlim(-1, 2)
ax2.set_ylim(1, 15)

ax1.tick_params(axis="x", rotation=30)
ax2.tick_params(axis="x", rotation=30)

ax1.set_xlabel("")
ax2.set_xlabel("")

ax1.legend_.remove()
ax2.legend_.remove()
ax1.set_ylabel("Estimated alpha values")
ax2.set_ylabel("Estimated beta values")
fig.fig.suptitle("Q-learning in two-armed bandit task")

C:\Users\asheshlab\Documents\Codes\NeuroPy\neuropy\plotting\figure.py:162: UserWarning: Ignoring specified arguments in this call because figure with num: 1 already exists
  fig = plt.figure(num=num, figsize=(8.5, 11), clear=True)


Text(0.5, 0.98, 'Q-learning in two-armed bandit task')